### Quick Run Block

In [1]:
# import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

# # Path to the problematic checkpoint
# checkpoint_path = r"c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\pre_trained_models\reid\dukemtmcreid_resnet50_256_128_epoch_120.ckpt"

# # Allow loading of ModelCheckpoint objects (since PyTorch 2.6 blocks this by default)
# torch.serialization.add_safe_globals([ModelCheckpoint])

# # Load the checkpoint safely
# checkpoint = torch.load(checkpoint_path, map_location="cpu", weights_only=False)

# # Print out keys to inspect conflicting ones
# print("Checkpoint keys:", checkpoint.keys())

# # Remove ModelCheckpoint states (if they exist)
# keys_to_remove = [key for key in checkpoint.keys() if "ModelCheckpoint" in key]
# for key in keys_to_remove:
#     del checkpoint[key]

# # Save the cleaned checkpoint
# fixed_checkpoint_path = checkpoint_path.replace(".ckpt", "_fixed.ckpt")
# torch.save(checkpoint, fixed_checkpoint_path)

# print(f"Fixed checkpoint saved to: {fixed_checkpoint_path}")

## Code

In [ ]:
import sys
from pathlib import Path
import os

sys.path.append(str(Path.cwd().parent.parent))
print(str(Path.cwd().parent.parent))
print("Current working directory: ", os.getcwd())

from ModelDevelopment.CentralPipeline import CentralPipeline
from ModelDevelopment.ImageBatchPipeline import ImageBatchPipeline
from DataProcessing.DataPreProcessing import DataPaths

c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition
Current working directory:  c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\ModelDevelopment\experiments


## Multithreaded Studies Research Findings
- ThreadPoolExecutor works best for CentralPipeline pre-processing step (i.e. everything before run_pose)
- Time taken to process one batch of 32 tracklets: 251 seconds => (251*32)/3600 = 2.35 hours ETA @ 200 images per batch cap, num_threads = 6x3 = 18
- Note on the GPU_SEMAPHORE constant inside ImageFeatureTransformPipeline: this is always capped at 1 so we only offload 1 batch of images to the GPU
- Since this gate is capped at 1, we can control the amount of data offloaded to the GPU via the image_batch_size param to CentralPipeline and only that
- Since caching is implemented, you can try setting the gate to 2 for even faster results, and if the process crashes, keep restarting with use_cache=True

In [ ]:
pipeline = CentralPipeline(
  #tracklets_to_process_override=["34"],
  num_tracklets=1210,
  #num_images_per_tracklet=50,
  input_data_path=DataPaths.TEST_DATA_DIR.value,
  output_processed_data_path=DataPaths.PROCESSED_DATA_OUTPUT_DIR_TEST.value,
  common_processed_data_dir=DataPaths.COMMON_PROCESSED_OUTPUT_DATA_TEST.value,
  gt_data_path=DataPaths.TEST_DATA_GT.value,
  single_image_pipeline=False,
  display_transformed_image_sample=False, # NOTE: DO NOT USE. Code is parallelized so we cannot show images anymore. Code breaks, but first one will show if True.
  num_image_samples=1,
  use_cache=True, # Set to false if you encounter data inconsistencies.
  suppress_logging=False,
  
  # --- PARALLELIZATION PARAMS --- These settings are optimal for an NVIDIA RTX 3070 Ti Laptop GPU.
  num_workers=2,            # CRITICAL optimisation param. Adjust accordingly. 6
  tracklet_batch_size=32,   # CRITICAL optimisation param. Adjust accordingly. 32
  image_batch_size=100,     # CRITICAL optimisation param. Adjust accordingly. 200
  num_threads_multiplier=2  # CRITICAL optimisation param. Adjust accordingly. 3. Interpretation: num_threads = num_workers * num_threads_multiplier
  )

2025-03-23 22:45:37 [INFO] DataPreProcessing initialized. Universe of available data paths:
2025-03-23 22:45:37 [INFO] ROOT_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted
2025-03-23 22:45:37 [INFO] TEST_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\test_gt.json
2025-03-23 22:45:37 [INFO] TRAIN_DATA_GT: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\train_gt.json
2025-03-23 22:45:37 [INFO] TEST_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\test\images
2025-03-23 22:45:37 [INFO] TRAIN_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\train\images
2025-03-23 22:45:37 [INFO] CHALLENGE_DATA_DIR: c:\Users\colin\OneDrive\Desktop\Jersey-Number-Recognition\data\SoccerNet\jersey-2023\extracted\challenge\images
2

In [ ]:
pipeline.run_soccernet(
  run_soccer_ball_filter=False,
  generate_features=False,
  run_filter=False,
  run_legible=False,
  run_legible_eval=True,
  run_pose=False,
  run_crops=False,
  run_str=False,
  run_combine=False,
  run_eval=False)

2025-03-23 22:45:37 [INFO] Running the SoccerNet pipeline.
2025-03-23 22:45:37 [INFO] Evaluating legibility results on 1210 tracklets


Evaluating legibility:   0%|          | 0/1210 [00:00<?, ?it/s]

2025-03-23 22:45:37 [INFO] FP: 39
2025-03-23 22:45:37 [INFO] FP: 61
2025-03-23 22:45:37 [INFO] FP: 87
2025-03-23 22:45:37 [INFO] FP: 89
2025-03-23 22:45:37 [INFO] FP: 97
2025-03-23 22:45:37 [INFO] FP: 134
2025-03-23 22:45:37 [INFO] FP: 162
2025-03-23 22:45:37 [INFO] FP: 164
2025-03-23 22:45:37 [INFO] FP: 167
2025-03-23 22:45:37 [INFO] FP: 168
2025-03-23 22:45:37 [INFO] FP: 172
2025-03-23 22:45:37 [INFO] FP: 199
2025-03-23 22:45:37 [INFO] FP: 215
2025-03-23 22:45:37 [INFO] FP: 260
2025-03-23 22:45:37 [INFO] FP: 267
2025-03-23 22:45:37 [INFO] FP: 269
2025-03-23 22:45:37 [INFO] FP: 276
2025-03-23 22:45:37 [INFO] FP: 281
2025-03-23 22:45:37 [INFO] FP: 288
2025-03-23 22:45:37 [INFO] FP: 295
2025-03-23 22:45:37 [INFO] FP: 317
2025-03-23 22:45:37 [INFO] FP: 325
2025-03-23 22:45:37 [INFO] FP: 340
2025-03-23 22:45:37 [INFO] FP: 345
2025-03-23 22:45:37 [INFO] FP: 374
2025-03-23 22:45:37 [INFO] FP: 386
2025-03-23 22:45:37 [INFO] FP: 388
2025-03-23 22:45:37 [INFO] FP: 391
2025-03-23 22:45:37 [INFO